In [36]:
import pandas
import pickle
import matplotlib as plt
import numpy as np
from patsy import *
import sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression



In [37]:
# Pickling functions
def pickle_it(data, filename):
    with open(filename, "wb") as picklefile:
        pickle.dump(data, picklefile)

def load_pickle(filename):
    with open(filename, "rb") as picklefile: 
        return pickle.load(picklefile)

In [38]:
d = load_pickle('fixyear.pkl')
d1 = load_pickle('fixyearfull.pkl')

In [39]:
len(d1)

451

In [40]:
df = pandas.DataFrame(d)
df.dtypes

Budget                float64
Gross                 float64
Metascore             float64
OpeningWkd            float64
ReleaseDate    datetime64[ns]
Runtime               float64
Title                  object
Year                  float64
dtype: object

In [41]:
y, X = dmatrices('Gross ~ Budget + Year + OpeningWkd + Runtime + Metascore', data = df, return_type = 'dataframe')

In [42]:
y

,Gross
38,189412677
87,1000878
104,23296694
145,83348920
166,196573705
196,35287788
203,4091
213,55747724
220,261437578
234,26687172


In [43]:
X

,Intercept,Budget,Year,OpeningWkd,Runtime,Metascore
38,1,165000000,2012,49038712,101,72
87,1,22000000,2011,57585,91,71
104,1,150000000,2008,63106589,89,61
145,1,132000000,2014,25447444,92,53
166,1,90000000,2009,41690382,94,50
196,1,39000000,2012,11412213,87,74
203,1,34000000,2013,4091,85,47
213,1,55000000,2013,15805237,91,38
220,1,92000000,2004,70467623,115,90
234,1,30000000,2011,159369,63,74


In [44]:
model = sm.OLS(y,X)

In [45]:
results = model.fit()

In [46]:
print type(model)
print type(results)

<class 'statsmodels.regression.linear_model.OLS'>
<class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>


In [47]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Gross   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     10.08
Date:                Tue, 06 Oct 2015   Prob (F-statistic):           1.67e-08
Time:                        21:50:36   Log-Likelihood:                -3692.4
No. Observations:                 183   AIC:                             7397.
Df Residuals:                     177   BIC:                             7416.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   6.287e+09   4.44e+09      1.414      0.159     -2.49e+09  1.51e+10
Budget        -0.0177      0.033     -0.530      0.597        -0.083     0.048
Year       -3.131e+06   2.22e+06     -1.408      0.161     -7.52e+06  1.26e+06
OpeningWkd     2.8932      0.443      6.534      0.000         2.019     3.767
Runtime     1.741e+05   1.17e+06      0.148      0.882     -2.14e+06  2.49e+06
Metascore   2.412e+05   6.58e+05      0.367      0.714     -1.06e+06  1.54e+06
==============================================================================
Omnibus:                      257.036   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22206.049
Skew:                           5.960   Prob(JB):                         0.00
Kurtosis:                      55.632   Cond. No.                     1.63e+11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.63e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [48]:
results = smf.ols(formula = "Gross ~ Budget + OpeningWkd + Runtime + Year + Metascore", data = df).fit()

In [49]:
print results.params

Intercept     6.286774e+09
Budget       -1.765267e-02
OpeningWkd    2.893248e+00
Runtime       1.740610e+05
Year         -3.131031e+06
Metascore     2.411551e+05
dtype: float64


In [50]:


model = LinearRegression()
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [51]:
model.score(X,y)

0.22157704216364571

In [52]:
model.coef_

array([[  0.00000000e+00,  -1.76526700e-02,  -3.13103095e+06,
          2.89324819e+00,   1.74060990e+05,   2.41155101e+05]])

In [53]:
model.intercept_

array([  6.28677367e+09])

In [62]:
import numpy as np
import pandas
from pandas import DataFrame, Series
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
import scipy, scipy.stats

df.plot.scatter(df.Gross, df.Budget, alpha = 0.5)

AttributeError: 'function' object has no attribute 'scatter'